In [1]:
#Spectral tools first experiments
#Based on José Luis Notebook on Spectral tools

In [2]:
import numpy as np
from math import pi, sqrt, cos, acos # adding cos and acos
from scipy import linalg as LA
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import ticker, cm
import cmath


In [3]:
##########################################################################################
# Chebyshev Interpolation routines
##########################################################################################

#Identical to José Luis'

def Chebyshev(m,x) :
    
    """
    Returns the value of the T_m(x)
    """
    return cos(m*acos(x))

# Modification:
# Comented out unnecessary variable

def Cheb_spectral_interpol(coeffs,x) :

    """
    Returns the Chebyshev interpolation at x\in[1,1], given the
    Chebishev coefficients coeffs
    """
    n = len(coeffs)
    #EG: this N does nothing in this function.  it is just a bookkeeping device, right?
    # EG:  N labels the x_N point and there are n=N+1 in the set {x_N}.
    #N = n-1
    n_grid = np.arange(0,n)
    
    temp=0
    for k in np.arange(1,n):
        temp += coeffs[k]*Chebyshev(k,x)
    
    interpol = 1./2.*coeffs[0] + temp
    
    return interpol

#########################################################################################
# Chebyshev basic spectral elements
##########################################################################################

##########################################################################################
#Chebyshev-Lobatto
##########################################################################################

#Modification: dictionary description changed

def CL_grid(N) : 
    
    """
    Implement collocation points of the Chebyshev-Lobatto grid
    x_j with j=0,1,...,N
    (following point 4.6.4. in Marcus' notes)
    
    It returns the collocation points
    """
    # EG: This function does not return the Differentiation Matrix, only the collocation points
    n = N+1
    n_grid = np.arange(0, n)
    x_CL = np.cos(pi*n_grid/N)
    
    return x_CL

#Identical to José Luis'

def coeffs_CL(f) :
    
    """ 
    Reads an ndarray with the values of the function at the Chebyshev-Lobatto
    collocation points and return the spectral coefficients
    (following point 4.3.4. in Marcus' notes)
    
    Note: f[N] corresponds to f(-1) and f[0]=1
    """
    
    n = len(f)
    N = n-1
    n_grid = np.arange(0,n)
    
    coeffs = np.zeros(n)
    print(coeffs)
    

    for i in n_grid :
        if i == N :
            temp = 0.
            for k in np.arange(1,N) :
                temp += f[k]*Chebyshev(i,CL_grid(N)[k])
            coeffs[i] = 1./(2.*N)*(f[0] + np.float_power((-1),i)*f[N] + 2.*temp)
           
            
        else :
            temp = 0
            for k in np.arange(1,N) :
                temp +=  f[k]*Chebyshev(i,CL_grid(N)[k])
            coeffs[i] = 1./N*(f[0] + np.float_power((-1),i)*f[N] + 2.*temp)
            
    return coeffs

#########################################################################################
# Spectral-Derivation Matrices
##########################################################################################

# All the remaining of the cell is identical to José Luis'

##########################################################################################
#Chebyshev-Lobatto
##########################################################################################


def D1_CL(N) :

    """
    Implement the matrix of the 1st Derivative
    associated with the Chebyshev-Lobatto grid,
    with grid points: x_j with j=0,1,...,N
    (following point 4.6.4. in Marcus' notes)
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_CL = np.cos(pi*n_grid/N)
    
    kappa = np.ones(n); kappa[0]=2.; kappa[N]=2.
    D1_CL = np.zeros((n,n))

    for i in n_grid :
        #Diagonal values
        D1_CL[0,0] = (2.*N**2+1.)/6.
        D1_CL[N,N] = -(2.*N**2+1.)/6.
        if ((i!=0) & (i!=N)) : D1_CL[i,i] = -x_CL[i]/(2*(1-x_CL[i]**2))
        #Out-of-diagonal values
        for j in n_grid :
            if (j!=i) : D1_CL[i,j] = kappa[i]/kappa[j]* (-1.)**(i-j)/(x_CL[i]-x_CL[j])
            #if (j!=i) : D1_CL[i,j] = (-1)**(i-j)    
    
    return x_CL, D1_CL


    
def D2_CL(N) :

    """
    Implement the matrix of the 2nd Derivative
    associated with the Chebyshev-Lobatto grid,
    with grid points: x_j with j=0,1,...,N
    (following point 4.6.4. in Marcus' notes)
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_CL = np.cos(pi*n_grid/N)
    
    kappa = np.ones(n); kappa[0]=2.; kappa[N]=2. 
    D2_CL = np.zeros((n,n))

    for i in n_grid :
        #Diagonal values
        D2_CL[0,0] = (N**4-1.)/15.
        D2_CL[N,N] = D2_CL[0,0]
        if ((i!=0) & (i!=N)) : 
            D2_CL[i,i] = -1./(1-x_CL[i]**2)**2 - (N**2-1.)/(3.*(1.-x_CL[i]**2))
        #Out-of-diagonal values
        for j in n_grid :
            if (j!=0) : 
                D2_CL[0,j] = 2./3. * (-1.)**j/kappa[j] * ((2.*N**2+1.)*(1-x_CL[j])-6.)/(1.-x_CL[j])**2
            if (j!=N) : 
                D2_CL[N,j] = 2./3. * (-1.)**(N+j)/kappa[j] * ((2.*N**2+1.)*(1+x_CL[j])-6.)/(1.+x_CL[j])**2
            if ((j!=i) & (i!=0) & (i!=N)) : 
                D2_CL[i,j] = (-1.)**(i-j)/kappa[j] * (x_CL[i]**2+x_CL[i]*x_CL[j]-2.)/((1-x_CL[i]**2)*(x_CL[j]-x_CL[i])**2)  

    return x_CL, D2_CL

##########################################################################################
#Chebyshev-Gauss
##########################################################################################


def D1_CG(N) :

    """
    Implement the matrix of the 1st Derivative
    associated with the Chebyshev-Gauss grid,
    with grid points: x_j with j=0,1,...,N
    (following point 4.6.3. in Marcus' notes)
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_CG = np.cos((pi*n_grid+0.5*pi)/(N+1))
    D1_CG = np.zeros((n,n))

    for i in n_grid :
        for j in n_grid :
            if (i==j) :   D1_CG[i,j] = (x_CG[i])/(2*(1-(x_CG[i])**2))
            if (i!=j) :   D1_CG[i,j]=(np.float_power((-1),(i-j)))/(x_CG[i]-x_CG[j])*sqrt((1-(x_CG[j])**2)/(1-x_CG[i]**2))
    
    return x_CG, D1_CG 


def D2_CG(N) :
    
    """
    Implement the matrix of the 2nd Derivative
    associated with the Chebyshev-Gauss grid,
    with grid points: x_j with j=0,1,...,N
    (following point 4.6.3. in Marcus' notes)
    
    
    It returns a tuple with the grid points in the
    first entry and the Differentation Matrix in the second entry
    """
    
    n = N+1
    n_grid = np.arange(0, n)
    x_CG = np.cos((pi*n_grid+pi*0.5)/(N+1))
    D2_CG = np.zeros((n,n))

    for i in n_grid :
        for j in n_grid :
            if (i==j) : D2_CG[i,j] = (x_CG[i])**2/(1-(x_CG[i])**2)**2-N*(N+2)/(3*(1-(x_CG[i])**2))
            else : D2_CG[i,j] = (np.float_power((-1),(i-j)))/(x_CG[i]-x_CG[j])*sqrt((1-x_CG[j]**2)/(1-x_CG[i]**2))*(((x_CG[i])/(1-x_CG[i]**2))-(2/(x_CG[i]-x_CG[j])))
    return x_CG, D2_CG   

In [5]:
#########################################################
# Spectrum calculator
#########################################################

# Modifications:  B is not used and N=n-1 instead of N= n/2 - 1
def Spectrum(M) :
    """
    Implement Spectrum of a matrix M.
    Returns a tuple with the real and imaginary part of the eigenvalues respectively.
    """
    from scipy import linalg as LA
    
    ### 1. Preparation of tools
    sizeM = np.shape(M)
    if  np.shape(M)[0]!= np.shape(M)[1]:
        print("Non-square Matrix!!! ")
    else : 
        n = np.shape(M)[0]
                       
        eigenvalues_M = LA.eigvals(M)
        eigenvalues_M_Re = eigenvalues_M.real
        eigenvalues_M_Im = eigenvalues_M.imag
        return eigenvalues_M
    
    # Modifications:  following José Luis'
def SpectrumAlt(L,B) :
    """
    Implement Spectrum of a matrix M.
    Returns a tuple with the real and imaginary part of the eigenvalues respectively.
    """
    from scipy import linalg as LA
    
    ### 1. Preparation of tools
    sizeL = np.shape(L)
    if  np.shape(L)[0]!= np.shape(L)[1]:
        print("Non-square Matrix!!! ")
    else : 
        n = np.shape(L)[0]
    N = n/2 - 1
    # EG: should't this be  n//2 -1 so that it gives an integer?
    # EG: this N is different from Rodrigo's notation/convention N=n-1
   
    ### 2. Calculation of the Spectrum (eigenvalues)
    #eigenvalues_L, eigenvectors_L = LA.eig(L,B)
    eigenvalues_L = LA.eigvals(L,B)
    #EG: what is the second argument doing?
    #EG: is this the generlised Eigenvalue problem?
    # EG: How/why is this used/needed for the pseudospectrum calculation?

    eigenvalues_L_Re = eigenvalues_L.real
    eigenvalues_L_Im = eigenvalues_L.imag
    
    return eigenvalues_L
    
    
    

    

        

In [ ]:
def Diagonal_Random_Matrix(n) :
    D = np.random.rand(n,1)
    Id = np.eye(n)
    DR = Id * D 
    #visualisation_matrix(DR)
    #DR = np.zeros(n,n)    
    #for i in range(n):
    #    DR[i,i] = D[i,i] 
    return DR